# 1. Load Packages

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold, RepeatedStratifiedKFold, GridSearchCV
import pandas as pd
import matplotlib.pylab as plt
from matplotlib import font_manager
from matplotlib.font_manager import FontProperties
from tqdm import tqdm
import shap
import time

# 2. Data Splitting

In [ ]:
df_train = pd.read_excel("../gandalf-doe/test_data/dataset.xlsx", index_col=0, sheet_name="Training")
df_train.head(1)

In [ ]:
df_test = pd.read_excel("../gandalf-doe/test_data/dataset.xlsx", index_col=0, sheet_name="Test")
df_test.head(1)

In [ ]:
all_samples = pd.concat([df_train, df_test]).reset_index(drop=True)
all_samples.head(1)

In [ ]:
# Determined by Mahalanobis distance

inter_train = all_samples.drop([5, 9, 12, 13, 16, 17, 21, 31, 37, 43])
inter_test = all_samples.loc[[5, 9, 12, 13, 16, 17, 21, 31, 37, 43]]

# 3. Data Normalization

In [ ]:
variables = ['Temperature', 'Pressure', 'GHSV', 'Ni', 'Co', 'Calcination', 'Reduction']

In [ ]:
scale_max = np.array([7.63e+02, 1.00e+01, 2.64e+04, 2.50e+01, 1.00e+01, 9.23e+02, 9.23e+02])
scale_min = np.array([5.23e+02, 1.00e+00, 3.30e+03, 0.00e+00, 0.00e+00, 6.23e+02, 6.23e+02])

# 4. Nested Repeated k-fold Cross-Validation

In [ ]:
xgb_param_grid = {
    'booster': ['gbtree'],                    # Booster type
    'n_estimators': [1000, 5000, 10000],
                                              # Number of trees
    'max_depth': [2, 3, 4],                   # Depth of each tree
    'learning_rate': [0.01, 0.05, 0.25, 0.5, 0.75],
                                              # Learning rate
    'subsample': [0.1, 0.5, 0.6, 0.7, 1],        
                                              # Fraction of samples used per tree
    'tree_method': ['hist', 'exact'],         # Method to grow trees
}

In [ ]:
rf_param_grid = {
    'n_estimators': [100, 500, 1000],         # Number of trees
    'max_depth': [None, 2, 5],                # Depth of each tree
    'max_features': ['sqrt', 'log2'],         # Maximal features
    'min_samples_split': [2, 5, 10],          # Minimal sample split
}

In [ ]:
def nested_cross_validation_conversion(model, param_grid, X, y, n_splits=8, n_repeats=4):
    start_t = time.time()
    new_t = start_t
    outer_results = {"INDEX": [], "PRED": [], "TRUE": []}

    outer_cv = RepeatedStratifiedKFold(n_splits=n_splits, random_state=120897, n_repeats=n_repeats)

    for train_index, test_index in outer_cv.split(X, np.digitize(y, np.percentile(y, np.arange(0, 100, 10)))):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        inner_cv = KFold(n_splits=n_splits, shuffle=True, random_state=210995)
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=inner_cv, scoring='r2', n_jobs=-1)
        grid_search.fit(X_train, y_train)

        best_model = grid_search.best_estimator_
        y_pred = np.clip(best_model.predict(X_test), 0, 100)
        print(y_pred)
        
        # Store the results
        outer_results["INDEX"] += list(test_index)
        outer_results["PRED"] += list(y_pred)
        outer_results["TRUE"] += list(y_test)
        print("Finished a round after ", time.time()-new_t)
        new_t = time.time()

    # Compute final metrics on the outer results
    res = pd.DataFrame(outer_results)
    true_data = res["TRUE"].to_numpy()
    pred_data = res["PRED"].to_numpy()

    mae = mean_absolute_error(true_data, pred_data)
    rmse = np.sqrt(mean_squared_error(true_data, pred_data))
    r2 = r2_score(true_data, pred_data)

    print(f"Nested CV Performance: MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.3f}")
    
    return mae, rmse, r2

In [ ]:
def nested_cross_validation_sty(model, param_grid, X, y, ghsv, n_splits=8, n_repeats=4):
    outer_results = {"INDEX": [], "PRED": [], "TRUE": []}

    outer_cv = RepeatedStratifiedKFold(n_splits=n_splits, random_state=120897, n_repeats=n_repeats)

    for train_index, test_index in outer_cv.split(X, np.digitize(y, np.percentile(y, np.arange(0, 100, 10)))):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        ghsv_test = ghsv[test_index]

        inner_cv = KFold(n_splits=n_splits, shuffle=True, random_state=210995)
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=inner_cv, scoring='r2', n_jobs=-1)
        grid_search.fit(X_train, y_train)

        best_model = grid_search.best_estimator_
        y_pred = np.clip(best_model.predict(X_test), 0, 100)
        print(y_pred)
        preds = (y_pred / 100) * ghsv_test * (12.011 + 4*1.0079) / (5*22400)
        trues = (y_test / 100) * ghsv_test * (12.011 + 4*1.0079) / (5*22400)
        
        # Store the results
        outer_results["INDEX"] += list(test_index)
        outer_results["PRED"] += list(preds)
        outer_results["TRUE"] += list(trues)

    # Compute final metrics on the outer results
    res = pd.DataFrame(outer_results)
    true_data = res["TRUE"].to_numpy()
    pred_data = np.clip(res["PRED"].to_numpy(), 0, 100)

    mae = mean_absolute_error(true_data, pred_data)
    rmse = np.sqrt(mean_squared_error(true_data, pred_data))
    r2 = r2_score(true_data, pred_data)

    print(f"Nested CV Performance: MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.3f}")
    
    return mae, rmse, r2

## 4.1 Conversion with XGB

In [ ]:
scaled_input = (df_train[variables].to_numpy() - scale_min) / (scale_max - scale_min)
nested_cross_validation_conversion(xgb.sklearn.XGBRegressor(random_state=0), xgb_param_grid, scaled_input, df_train["Conversion"].to_numpy(), n_splits=8, n_repeats=4)

## 4.2 Conversion with RF

In [ ]:
scaled_input = (df_train[variables].to_numpy() - scale_min) / (scale_max - scale_min)
nested_cross_validation_conversion(RandomForestRegressor(random_state=0), rf_param_grid, scaled_input, df_train["Conversion"].to_numpy(), n_splits=8, n_repeats=4)

## 4.3 STY with XGBoost

In [ ]:
scaled_input = (df_train[variables].to_numpy() - scale_min) / (scale_max - scale_min)
nested_cross_validation_sty(xgb.sklearn.XGBRegressor(random_state=0), xgb_param_grid, scaled_input, df_train["Conversion"].to_numpy(), df_train["GHSV"].to_numpy(), n_splits=8, n_repeats=4)

## 4.4 STY with RF

In [ ]:
scaled_input = (df_train[variables].to_numpy() - scale_min) / (scale_max - scale_min)
nested_cross_validation_sty(RandomForestRegressor(random_state=0), rf_param_grid, scaled_input, df_train["Conversion"].to_numpy(), df_train["GHSV"].to_numpy(), n_splits=8, n_repeats=4)

# 5. Interpolation Testing

In [ ]:
def test_external_dataset_on_conversion(model, X_train, y_train, X_test, y_test, param_grid, n_splits):
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=210995)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring='r2', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    print("Best params", grid_search.best_params_)
    y_pred = np.clip(best_model.predict(X_test), 0, 100)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    return mae, rmse, r2

In [ ]:
def test_external_dataset_on_sty(model, X_train, y_train, X_test, y_test, param_grid, n_splits, ghsv):
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=210995)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring='r2', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    print("Best params", grid_search.best_params_)
    y_pred = np.clip(best_model.predict(X_test), 0, 100)
    preds = (y_pred / 100) * ghsv * (12.011 + 4*1.0079) / (5*22400)
    trues = (y_test / 100) * ghsv * (12.011 + 4*1.0079) / (5*22400)
                                
    mae = mean_absolute_error(trues, preds)
    rmse = np.sqrt(mean_squared_error(trues, preds))
    r2 = r2_score(trues, preds)
    return mae, rmse, r2

In [ ]:
start_time = time.time()
scaled_train_input = (inter_train[variables].to_numpy() - scale_min) / (scale_max - scale_min)
scaled_test_input = (inter_test[variables].to_numpy() - scale_min) / (scale_max - scale_min)
test_external_dataset_on_conversion(xgb.sklearn.XGBRegressor(random_state=0), 
                      scaled_train_input, inter_train["Conversion"].to_numpy(), 
                      scaled_test_input, inter_test["Conversion"].to_numpy(), 
                      xgb_param_grid, 8)

In [ ]:
scaled_train_input = (inter_train[variables].to_numpy() - scale_min) / (scale_max - scale_min)
scaled_test_input = (inter_test[variables].to_numpy() - scale_min) / (scale_max - scale_min)
test_external_dataset_on_conversion(RandomForestRegressor(random_state=0), 
                      scaled_train_input, inter_train["Conversion"].to_numpy(), 
                      scaled_test_input, inter_test["Conversion"].to_numpy(), 
                      rf_param_grid, 8)

In [ ]:
scaled_train_input = (inter_train[variables].to_numpy() - scale_min) / (scale_max - scale_min)
scaled_test_input = (inter_test[variables].to_numpy() - scale_min) / (scale_max - scale_min)
test_external_dataset_on_sty(xgb.sklearn.XGBRegressor(random_state=0), 
                             scaled_train_input, inter_train["Yield"].to_numpy(), 
                             scaled_test_input, inter_test["Yield"].to_numpy(), 
                             xgb_param_grid, 8, inter_test["GHSV"].to_numpy())

In [ ]:
scaled_train_input = (inter_train[variables].to_numpy() - scale_min) / (scale_max - scale_min)
scaled_test_input = (inter_test[variables].to_numpy() - scale_min) / (scale_max - scale_min)
test_external_dataset_on_sty(RandomForestRegressor(random_state=0), 
                             scaled_train_input, inter_train["Yield"].to_numpy(), 
                             scaled_test_input, inter_test["Yield"].to_numpy(), 
                             rf_param_grid, 8, inter_test["GHSV"].to_numpy())

# 6. Extrapolation Testing

In [ ]:
scaled_train_input = (df_train[variables].to_numpy() - scale_min) / (scale_max - scale_min)
scaled_test_input = (df_test[variables].to_numpy() - scale_min) / (scale_max - scale_min)
test_external_dataset_on_conversion(xgb.sklearn.XGBRegressor(random_state=0), 
                                    scaled_train_input, df_train["Conversion"].to_numpy(), 
                                    scaled_test_input, df_test["Conversion"].to_numpy(), 
                                    xgb_param_grid, 8)

In [ ]:
scaled_train_input = (df_train[variables].to_numpy() - scale_min) / (scale_max - scale_min)
scaled_test_input = (df_test[variables].to_numpy() - scale_min) / (scale_max - scale_min)
test_external_dataset_on_co nversion(RandomForestRegressor(random_state=0), 
                                    scaled_train_input, df_train["Conversion"].to_numpy(), 
                                    scaled_test_input, df_test["Conversion"].to_numpy(), 
                                    rf_param_grid, 8)

In [ ]:
scaled_train_input = (df_train[variables].to_numpy() - scale_min) / (scale_max - scale_min)
scaled_test_input = (df_ b  test[variables].to_numpy() - scale_min) / (scale_max - scale_min)
test_external_dataset_on_sty(xgb.sklearn.XGBRegressor(random_state=0), 
                             scaled_train_input, df_train["Yield"].to_numpy(), 
                             scaled_test_input, df_test["Yield"].to_numpy(), 
                             xgb_param_grid, 8, df_test["GHSV"].to_numpy())

In [ ]:
scaled_train_input = (df_train[variables].to_numpy() - scale_min) / (scale_max - scale_min)
scaled_test_input = (df_test[variables].to_numpy() - scale_min) / (scale_max - scale_min)
test_external_dataset_on_sty(RandomForestRegressor(random_state=0), 
                             scaled_train_input, df_train["Yield"].to_numpy(), 
                             scaled_test_input, df_test["Yield"].to_numpy(), 
                             rf_param_grid, 8, df_test["GHSV"].to_numpy())

# 7. SHAP

In [ ]:
def nested_cross_validation_shap(model, param_grid, X, y, n_splits=8, n_repeats=4):
    feature_importances = None
    outer_results = {"INDEX": [], "PRED": [], "TRUE": []}

    outer_cv = RepeatedStratifiedKFold(n_splits=n_splits, random_state=120897, n_repeats=n_repeats)

    for train_index, test_index in outer_cv.split(X, np.digitize(y, np.percentile(y, np.arange(0, 100, 10)))):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        inner_cv = KFold(n_splits=n_splits, shuffle=True, random_state=210995)
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=inner_cv, scoring='r2', n_jobs=-1)
        grid_search.fit(X_train, y_train)

        best_model = grid_search.best_estimator_
        y_pred = np.clip(best_model.predict(X_test), 0, 100)
        print(y_pred)
        outer_results["INDEX"] += list(test_index)
        outer_results["PRED"] += list(y_pred)
        outer_results["TRUE"] += list(y_test)
        explainer = shap.TreeExplainer(best_model)
        explanation = explainer(X_train, check_additivity=False)
        feature_importance = np.mean(np.abs(explanation.values), axis=0)
        feature_importance = feature_importance / np.sum(feature_importance)
        if feature_importances is None:
            feature_importances = feature_importance
        else:
            feature_importances = np.vstack((feature_importances, feature_importance))

    # Compute final metrics on the outer results
    res = pd.DataFrame(outer_results)
    true_data = res["TRUE"].to_numpy()
    pred_data = res["PRED"].to_numpy()

    mae = mean_absolute_error(true_data, pred_data)
    rmse = np.sqrt(mean_squared_error(true_data, pred_data))
    r2 = r2_score(true_data, pred_data)
    
    avg_importances = np.mean(feature_importances, axis=0)
    sd_importances = np.std(feature_importances, axis=0)

    print(f"Nested CV Performance: MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.3f}")
    
    return avg_importances, sd_importances

In [ ]:
scaled_input = (df_train[variables].to_numpy() - scale_min) / (scale_max - scale_min)
x_avgs, x_sds = nested_cross_validation_shap(xgb.sklearn.XGBRegressor(random_state=0), xgb_param_grid, scaled_input, df_train["Conversion"].to_numpy(), n_splits=8, n_repeats=4)
print("Average importance:", x_avgs)
print("Standard deviation:", x_sds)

In [ ]:
scaled_input = (df_train[variables].to_numpy() - scale_min) / (scale_max - scale_min)
s_avgs, s_sds = nested_cross_validation_shap(xgb.sklearn.XGBRegressor(random_state=0), xgb_param_grid, scaled_input, df_train["CH4 Selectivity"].to_numpy(), n_splits=8, n_repeats=4)
print("Average importance:", s_avgs)
print("Standard deviation:", s_sds)

In [ ]:
scaled_input = (df_train[variables].to_numpy() - scale_min) / (scale_max - scale_min)
sty_avgs, sty_sds = nested_cross_validation_shap(xgb.sklearn.XGBRegressor(random_state=0), xgb_param_grid, scaled_input, df_train["STY"].to_numpy(), n_splits=8, n_repeats=4)
print("Average importance:", sty_avgs)
print("Standard deviation:", sty_sds)